In [1]:
!pip install transformers accelerate sentencepiece bitsandbytes langchain faiss-cpu huggingface_hub


In [2]:
!pip install langchain_huggingface

In [7]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4503276 sha256=a5d14c93f75d7f0b2a754cb97f5e490937576938f99aef89e11c34148d28420d
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [41]:
import os
import json
from pathlib import Path
from typing import List, Dict
import numpy as np
import faiss
import pickle
from tqdm.auto import tqdm


from langchain_huggingface import HuggingFaceEmbeddings

from llama_cpp import Llama

In [10]:

import zipfile
from pathlib import Path

zip_path_1 = "/content/Diagnosis_flowchart.zip"
zip_path_2 = "/content/Finished.zip"

extract_to_1 = "/content/Diagnosis_flowchart"
extract_to_2 = "/content/Finished"


Path(extract_to_1).mkdir(parents=True, exist_ok=True)
with zipfile.ZipFile(zip_path_1, 'r') as zip_ref:
    zip_ref.extractall(extract_to_1)
print(f"Files extracted to: {extract_to_1}")
Path(extract_to_2).mkdir(parents=True, exist_ok=True)
with zipfile.ZipFile(zip_path_2, 'r') as zip_ref:
    zip_ref.extractall(extract_to_2)
print(f"Files extracted to: {extract_to_2}")


Files extracted to: /content/Diagnosis_flowchart
Files extracted to: /content/Finished


In [42]:
NOTES_DIR = "/content/Finished"
KG_DIR = "/content/Diagnosis_flowchart"

EMBED_MODEL_NAME = "BAAI/bge-base-en-v1.5"
FAISS_INDEX_PATH = "faiss_index.index"
METADATA_PATH = "metadata.pkl"

CHUNK_SIZE = 180
CHUNK_OVERLAP = 40
TOP_K = 5


In [43]:
def list_json_files(dirpath: str) -> List[str]:
    return [str(p) for p in Path(dirpath).rglob("*.json")]

def split_key_type(raw_key: str):
    if "$" in raw_key:
        return raw_key.rsplit("$", 1)
    return raw_key, ""

def parse_single_note(note_path: str) -> List[Dict]:
    with open(note_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    chunks = []

    input_sections = {k: v for k,v in data.items() if k.lower().startswith("input")}
    raw_text = "\n".join(input_sections.values())
    note_id = Path(note_path).stem

    for k,v in input_sections.items():
        chunks.append({
            "text": f"{k}: {v}",
            "meta": {"file": note_path, "note_id": note_id}
        })

    def walk(node_dict, path=[]):
        for key, val in node_dict.items():
            content, ntype = split_key_type(key)
            cur_path = path + [content]
            if isinstance(val, dict) and val:
                walk(val, cur_path)
            else:
                chunks.append({
                    "text": content,
                    "meta": {"file": note_path, "note_id": note_id, "path": cur_path}
                })

    tree_nodes = {k:v for k,v in data.items() if not k.lower().startswith("input")}
    walk(tree_nodes)
    return chunks

def parse_kg_file(kg_path: str) -> List[Dict]:
    with open(kg_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    disease_name = Path(kg_path).stem
    chunks = []

    def flatten(dct, parent=""):
        for k,v in dct.items():
            path = f"{parent} → {k}" if parent else k
            if isinstance(v, str):
                chunks.append({
                    "text": v,
                    "meta": {"disease": disease_name, "type": k, "path": path, "file": kg_path}
                })
            elif isinstance(v, dict):
                flatten(v, path)
    flatten(data.get("knowledge", {}))
    return chunks

def chunk_text(text: str, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    words = text.split()
    if len(words) <= chunk_size:
        return [text]
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunks.append(" ".join(words[start:end]))
        start += chunk_size - overlap
    return chunks

def prepare_chunks(chunks: List[Dict]):
    docs, metas = [], []
    for c in chunks:
        for sc in chunk_text(c["text"]):
            docs.append(sc)
            metas.append(c["meta"])
    return docs, metas

In [44]:
all_chunks = []
for f in list_json_files(NOTES_DIR):
  all_chunks.extend(parse_single_note(f))
for f in list_json_files(KG_DIR):
  all_chunks.extend(parse_kg_file(f))

docs, metas = prepare_chunks(all_chunks)


In [45]:
import random

def inspect_chunks(docs: List[str], metas: List[Dict], num_samples: int = 5):
    assert len(docs) == len(metas), "Mismatch between docs and metas!"

    print(f"Total chunks: {len(docs)}")
    print(f"Inspecting {num_samples} random samples...\n")

    indices = random.sample(range(len(docs)), min(num_samples, len(docs)))

    for i, idx in enumerate(indices):
        print(f"--- Sample {i+1} (Index {idx}) ---")
        print("Text chunk preview:")
        print(docs[idx][:500] + ("..." if len(docs[idx]) > 500 else ""))
        print("\nMetadata:")
        for k, v in metas[idx].items():
            print(f"  {k}: {v}")
        print("\n" + "-"*60 + "\n")


In [46]:
inspect_chunks(docs, metas, num_samples=5)


Total chunks: 9020
Inspecting 5 random samples...

--- Sample 1 (Index 5918) ---
Text chunk preview:
starting just past takeoff of L subclavian artery and extending down to above iliac bifurcation

Metadata:
  file: /content/Finished/Finished/Aortic Dissection/Type B Aortic Dissection/14210167-DS-14.json
  note_id: 14210167-DS-14
  path: ['Type B Aortic Dissection', 'Confirmed aortic dissection, beginning distal to the left subclavian artery and extending to the area above the iliac bifurcation', 'starting just past takeoff of L subclavian artery and extending down to above iliac bifurcation']

------------------------------------------------------------

--- Sample 2 (Index 8110) ---
Text chunk preview:
from around the time when he began to have difficulty speaking his R hand has felt more numb and clumsy than usual.

Metadata:
  file: /content/Finished/Finished/Stroke/Hemorrhagic Stroke/11408894-DS-2.json
  note_id: 11408894-DS-2
  path: ['Hemorrhagic Stroke', 'Suspected Stroke', 'Nu

In [47]:
from tqdm import tqdm
import numpy as np
import faiss
import pickle
from langchain_huggingface import HuggingFaceEmbeddings
from typing import List, Dict

hf_embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)

def build_faiss_index(docs: List[str], metas: List[Dict]):
    print("Embedding documents with progress...")
    embeddings = []

    for doc in tqdm(docs, desc="Embedding docs"):
        emb = hf_embedder.embed_documents([doc])[0]
        embeddings.append(emb)

    embeddings = np.array(embeddings).astype("float32")
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    faiss.write_index(index, FAISS_INDEX_PATH)
    with open(METADATA_PATH, "wb") as f:
        pickle.dump(metas, f)

    print("FAISS index built and saved.")
    return index

def retrieve_chunks(query: str, index, docs, top_k=TOP_K):
    print("Embedding query...")
    q_emb = np.array(hf_embedder.embed_query(query)).astype("float32").reshape(1, -1)
    D, I = index.search(q_emb, top_k)

    with open(METADATA_PATH, "rb") as f:
        metas = pickle.load(f)

    results = []
    for dist, idx in zip(D[0], I[0]):
        results.append({
            "chunk": docs[idx],
            "metadata": metas[idx],
            "distance": float(dist)
        })

    return results


In [48]:
if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(METADATA_PATH):
    faiss_index = faiss.read_index(FAISS_INDEX_PATH)
    print("Loaded existing FAISS index.")
else:
    faiss_index = build_faiss_index(docs, metas)

Loaded existing FAISS index.


In [49]:
faiss_index_file = "faiss_index.index"
faiss.write_index(faiss_index, faiss_index_file)
print(f"FAISS index saved to {faiss_index_file}")

#to load:
# index = faiss.read_index(faiss_index_file)
# print("FAISS index loaded from disk")

FAISS index saved to faiss_index.index


In [50]:
import numpy as np

def verify_faiss_index(docs: list, index, top_k_self=5, top_k_query=3, sample_idx=0, sample_query="What are the symptoms of hypertension?"):
    """
    Verify FAISS index without re-embedding.

    Parameters:
    - docs: list of text chunks (aligned with FAISS index)
    - index: FAISS index
    - top_k_self: number of neighbors for self-retrieval
    - top_k_query: number of neighbors for sample query retrieval
    - sample_idx: index of chunk for self-retrieval test
    - sample_query: a query string for retrieval test
    """
    print("===== FAISS INDEX VERIFICATION =====\n")

    print(f"Number of chunks in memory: {len(docs)}")
    if index.ntotal != len(docs):
        print(f"WARNING: FAISS index has {index.ntotal} entries, but docs length is {len(docs)}!")
    else:
        print("FAISS index size matches number of chunks.")

    print("\n--- Self-Retrieval Test ---")
    query_vec = np.array([index.reconstruct(sample_idx)]).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vec, top_k_self)
    print(f"Query chunk (first 200 chars): {docs[sample_idx][:200]}...\n")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. Retrieved idx: {idx}, distance: {distances[0][i]:.4f}")
        print(f"   Chunk snippet: {docs[idx][:200]}...\n")

    sample_query = "What are the symptoms of Type I Diabetes?"
    query_emb = np.array(hf_embedder.embed_query(sample_query)).astype("float32").reshape(1, -1)
    distances, indices = faiss_index.search(query_emb, top_k_query)

    print("--- Sample Query Retrieval ---")
    print(f"Query: {sample_query}\n")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. Retrieved idx: {idx}, distance: {distances[0][i]:.4f}")
        print(f"   Chunk snippet: {docs[idx][:200]}...\n")


    print("\n===== END OF VERIFICATION =====\n")


In [13]:
verify_faiss_index(
    docs=docs,
    index=faiss_index,
    top_k_self=5,
    top_k_query=3,
    sample_idx=0,
    sample_query="What are the symptoms of Type I Diabetes?"
)

===== FAISS INDEX VERIFICATION =====

Number of chunks in memory: 9020
✅ FAISS index size matches number of chunks.

--- Self-Retrieval Test ---
Query chunk (first 200 chars): input1: malaise, melanotic stools
...

1. Retrieved idx: 0, distance: 0.0000
   Chunk snippet: input1: malaise, melanotic stools
...

2. Retrieved idx: 8571, distance: 0.2886
   Chunk snippet: input1: dark stools
...

3. Retrieved idx: 8634, distance: 0.2886
   Chunk snippet: input1: dark stools
...

4. Retrieved idx: 10, distance: 0.3207
   Chunk snippet: melanotic stools...

5. Retrieved idx: 7027, distance: 0.3589
   Chunk snippet: input1: nausea, vomiting, malaise
...

--- Sample Query Retrieval ---
Query: What are the symptoms of Type I Diabetes?

1. Retrieved idx: 8983, distance: 0.5089
   Chunk snippet: increased thirst; frequent urination; unexplained weight loss; increased hunger; blurry vision; numbness or tingling in the feet or hands; sores that do not heal; extreme fatigue; etc....

2. Retrieved idx:

In [51]:
MODEL_ID = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"

CHUNK_SIZE = 180
CHUNK_OVERLAP = 40
TOP_K = 5

In [70]:
!pip install -U bitsandbytes


In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


In [53]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
generation_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
def build_prompt(query: str, retrieved_chunks: List[Dict]):
    context_text = "\n".join([f"- {c['chunk']}" for c in retrieved_chunks])
    prompt = f"""[INST] <<SYS>>
You are a medical assistant trained on clinical reasoning data. Given the following patient query and related clinical observations, generate a diagnostic explanation or suggestion based on the context.
<</SYS>>

### Patient Query:
{query}

### Clinical Context:
{context_text}

### Diagnostic Explanation:
[/INST]
"""
    return prompt

def generate_local_answer(prompt, max_new_tokens=512):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    output = generation_model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        temperature=0.5,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.split("### Diagnostic Explanation:")[-1].strip()

def rag_pipeline(query: str, index, docs, top_k=TOP_K):
    retrieved = retrieve_chunks(query, index, docs, top_k=top_k)
    prompt = build_prompt(query, retrieved)
    answer = generate_local_answer(prompt)
    return {"query": query, "retrieved_chunks": retrieved, "answer": answer}


In [27]:
example_queries = [
    "What are the symptoms of Type I Diabetes?",
    "Which patients have hypertension as a symptom?",
    "Describe the diagnostic process for stroke."
]

for q in example_queries:
    result = rag_pipeline(q, faiss_index, docs, top_k=3)
    print(f"\nQUERY: {q}")
    print("RETRIEVED CHUNKS:")
    for i, r in enumerate(result['retrieved_chunks']):
        print(f"{i+1}. {r['chunk'][:200]}... (Distance: {r['distance']:.4f})")
    print("\nGENERATED ANSWER:")
    print(result['answer'])
    print("="*80)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Embedding query...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.



QUERY: What are the symptoms of Type I Diabetes?
RETRIEVED CHUNKS:
1. increased thirst; frequent urination; unexplained weight loss; increased hunger; blurry vision; numbness or tingling in the feet or hands; sores that do not heal; extreme fatigue; etc.... (Distance: 0.5089)
2. - mother  - type I diabetes... (Distance: 0.5595)
3. Diabetes... (Distance: 0.5671)

GENERATED ANSWER:
[/INST]
Type 1 diabetes, also known as juvenile diabetes or insulin-dependent diabetes, is an autoimmune disease where the body's immune system destroys the insulin-producing cells in the pancreas. This results in high blood sugar levels as insulin is required to regulate the amount of glucose in the bloodstream. The symptoms of Type 1 diabetes include increased thirst, frequent urination, unexplained weight loss, increased hunger, blurry vision, numbness or tingling in the feet or hands, sores that do not heal, and extreme fatigue. These symptoms can indicate that the patient's body is not producing enough i

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.



QUERY: Which patients have hypertension as a symptom?
RETRIEVED CHUNKS:
1. Patient reports a history of hypertension... (Distance: 0.3920)
2. +hypertension... (Distance: 0.4655)
3. +Hypertension... (Distance: 0.4655)

GENERATED ANSWER:
[/INST]

Based on the clinical context provided, patients with hypertension as a symptom are likely to have a medical history that includes high blood pressure. In this case, the patient has reported a history of hypertension and the clinical observations indicate the presence of hypertension. Therefore, the diagnostic suggestion is that the patient likely has hypertension, which is a common condition affecting many individuals.
Embedding query...

QUERY: Describe the diagnostic process for stroke.
RETRIEVED CHUNKS:
1. STROKE... (Distance: 0.4204)
2. Stroke... (Distance: 0.4204)
3. ischemic stroke... (Distance: 0.5428)

GENERATED ANSWER:
[/INST]
A stroke, also known as a cerebrovascular accident (CVA), is a medical emergency that occurs when the blood s

In [28]:
!pip install gradio

In [29]:
import gradio as gr
from typing import List, Dict

def rag_gradio_interface(query: str, top_k: int = 5):
    """
    Gradio wrapper for RAG pipeline.
    Returns retrieved chunks and generated answer.
    """
    result = rag_pipeline(query, faiss_index, docs, top_k=top_k)

    retrieved_texts = ""
    for i, r in enumerate(result['retrieved_chunks']):
        retrieved_texts += f"{i+1}. {r['chunk'][:500]}... (Distance: {r['distance']:.4f})\n\n"

    return retrieved_texts, result['answer']

with gr.Blocks() as demo:
    gr.Markdown("## 🩺 Clinical RAG Assistant")
    gr.Markdown(
        "Enter a patient query, and the model will return relevant clinical context "
        "from your FAISS knowledge base and generate a diagnostic explanation."
    )

    with gr.Row():
        query_input = gr.Textbox(label="Patient Query", placeholder="e.g. What are the symptoms of Type I Diabetes?", lines=2)
        top_k_slider = gr.Slider(minimum=1, maximum=10, step=1, value=5, label="Number of Retrieved Chunks (Top-K)")

    with gr.Row():
        retrieved_output = gr.Textbox(label="Retrieved Chunks", lines=10)
        answer_output = gr.Textbox(label="Generated Answer", lines=10)

    submit_btn = gr.Button("Run RAG")

    submit_btn.click(
        fn=rag_gradio_interface,
        inputs=[query_input, top_k_slider],
        outputs=[retrieved_output, answer_output]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://939100c58fc075d718.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [76]:
disease_map = {
    "What are the symptoms of Type I Diabetes?": "Type I Diabetes",
    "What are the symptoms of Type II Diabetes?": "Type II Diabetes",
    "What are the risk factors and symptoms of Alzheimer?": "Alzheimer",
    "How is Type I Diabetes diagnosed?": "Type I Diabetes",
    "Describe clinical assessment for Alzheimer patients": "Alzheimer"
}


In [97]:
def build_retrieval_ground_truth_from_index(docs, metas, queries):
    ground_truth = {}
    for query in queries:
        gt_indices = []
        q_lower = query.lower()
        for idx, meta in enumerate(metas):
            disease = meta.get("disease", "").lower()
            path = " ".join(meta.get("path", [])).lower()
            if any(word in disease or word in path for word in q_lower.split()):
                gt_indices.append(idx)
        ground_truth[query] = gt_indices
    return ground_truth


In [98]:
retrieval_gt = build_retrieval_ground_truth_from_index(docs, metas, list(disease_map.keys()))
for q, idxs in retrieval_gt.items():
    print(q, len(idxs))


What are the symptoms of Type I Diabetes? 5233
What are the symptoms of Type II Diabetes? 4275
What are the risk factors and symptoms of Alzheimer? 4821
How is Type I Diabetes diagnosed? 5233
Describe clinical assessment for Alzheimer patients 1330


In [83]:
import numpy as np

def evaluate_retrieval(queries, index, docs, metas, retrieval_gt, top_k=5):
    precisions, recalls = [], []

    print("===== RETRIEVAL EVALUATION =====\n")

    for query in queries:
        q_emb = np.array(hf_embedder.embed_query(query)).astype("float32").reshape(1, -1)
        D, I = index.search(q_emb, top_k)

        gt_indices = set(retrieval_gt.get(query, []))
        if not gt_indices:
            print(f"No ground truth found for query: {query}")
            continue

        retrieved_indices = set(I[0])
        true_positives = len(retrieved_indices & gt_indices)
        precision = true_positives / top_k
        recall = true_positives / len(gt_indices)

        precisions.append(precision)
        recalls.append(recall)

        print(f"Query: {query}")
        print(f"Precision@{top_k}: {precision:.3f}, Recall@{top_k}: {recall:.3f}\n")

    if precisions:
        print(f"Average Precision@{top_k}: {np.mean(precisions):.3f}")
        print(f"Average Recall@{top_k}: {np.mean(recalls):.3f}")
    print("===== END RETRIEVAL EVALUATION =====\n")


In [84]:

queries = list(disease_map.keys())
top_k = 5

evaluate_retrieval(
    queries=queries,
    index=faiss_index,
    docs=docs,
    metas=metas,
    retrieval_gt=retrieval_gt,
    top_k=top_k
)

===== RETRIEVAL EVALUATION =====

Query: What are the symptoms of Type I Diabetes?
Precision@5: 1.000, Recall@5: 0.001

Query: What are the symptoms of Type II Diabetes?
Precision@5: 1.000, Recall@5: 0.001

Query: What are the risk factors and symptoms of Alzheimer?
Precision@5: 0.200, Recall@5: 0.000

Query: How is Type I Diabetes diagnosed?
Precision@5: 1.000, Recall@5: 0.001

Query: Describe clinical assessment for Alzheimer patients
Precision@5: 0.400, Recall@5: 0.002

Average Precision@5: 0.720
Average Recall@5: 0.001
===== END RETRIEVAL EVALUATION =====

